MediaSorter V1 : Scan a folder for movies and detect if some of them are made by the same director

---

In [1]:
#!pip install tmdbsimple pandas tqdm

In [ ]:
import os
import re
import pandas as pd
import tmdbsimple as tmdb
from collections import defaultdict

#Replace with the TMDb API key
tmdb.API_KEY = 'YOUR_TMDB_API_KEY_HERE'

#Replace with the path to some media folder
MOVIES_FOLDER_PATH  = r"C:\Your\Media\Folder"

### Core fct

In [3]:
def clean_name(name):
    """
    Cleans a file or folder name to extract the movie/series title.
    Example: "The Godfather (1972) [FRENCH] [BRRIP]" -> "The Godfather"
    """
    # Remove file extensions (e.g., .mkv, .mp4)
    name = os.path.splitext(name)[0]
    
    # Remove the year in parentheses or brackets (e.g., (2020), [2020])
    name = re.sub(r'\s*[\(\[].*?[\)\]]', '', name).strip()
    
    # Remove common tags (e.g., 1080p, BluRay, FRENCH, VOSTFR, S01E01)
    name = re.sub(r'\s*(1080p|720p|HD|BluRay|FRENCH|VOSTFR|S\d+E\d+).*', '', name, flags=re.IGNORECASE).strip()

    return name

def search_director(title):
    """
    Searches the movie/series on TMDb and returns the director.
    """
    if not title:
        return None

    try:
        # 1. Search for the title
        search = tmdb.Search()
        response = search.movie(query=title)
        
        # If no result, try searching TV series
        if not response['results']:
            response = search.tv(query=title)
            if not response['results']:
                return None
        
        # 2. Get the first result
        media_id = response['results'][0]['id']
        media_type = response['results'][0]['media_type'] if 'media_type' in response['results'][0] else ('movie' if 'title' in response['results'][0] else 'tv')

        # 3. Get credits to extract the director
        if media_type == 'movie':
            movie = tmdb.Movies(media_id)
            credits = movie.credits()
            
            # The director is in the 'crew' section
            for crew_member in credits.get('crew', []):
                if crew_member.get('job') == 'Director':
                    return crew_member.get('name')
        
        # TV series logic is different (often multiple directors/showrunners)
        # We simplify by focusing on movies, as director info is clearer.
        if media_type == 'tv':
            return "TV Series (not analyzed)"

    except Exception as e:
        return "API search error"
        
    return "Not found"

### Scnanning fct

In [4]:
def scan_and_group_by_director(folder_path):
    
    movies_by_director = defaultdict(list)
    raw_results = []
    processed_titles = set()

    for current_path, subfolders, files in os.walk(folder_path):
        # We use folder names as they tend to be cleaner
        # We also analyze files so nothing is missed
        
        items_to_analyze = subfolders + files
        
        for name in items_to_analyze:
            cleaned_title = clean_name(name)
            
            # Avoid processing the same title more than once
            if cleaned_title and cleaned_title not in processed_titles:
                print(f"🔍 Processing title: {cleaned_title}...")
                
                director = search_director(cleaned_title)
                
                raw_results.append({
                    'Original name': name,
                    'Cleaned title': cleaned_title,
                    'Director': director,
                })
                
                # Group movies only when a valid director is found
                if director and director not in ["Not found", "API search error", "TV Series (not analyzed)"]:
                    movies_by_director[director].append(cleaned_title)
                    
                processed_titles.add(cleaned_title)

    return pd.DataFrame(raw_results), movies_by_director

### Main

In [ ]:
if os.path.exists(MOVIES_FOLDER_PATH):
    print(f"🎬 Starting scan and online search in: {MOVIES_FOLDER_PATH}\n")
    
    # Run the main function
    df_raw, grouped_movies = scan_and_group_by_director(MOVIES_FOLDER_PATH)
    
    print("\n" + "="*50)
    print("RESULTS OF DIRECTOR GROUPING")
    print("="*50)
    
    # Filter only directors with more than one movie
    multi_movie_directors = {
        director: movies 
        for director, movies in grouped_movies.items() 
        if len(movies) > 1
    }
    
    if multi_movie_directors:
        print(f"  {len(multi_movie_directors)} director(s) with multiple movies in your collection:\n")
        
        for director, movies in multi_movie_directors.items():
            print(f"--- **{director}** ({len(movies)} movies) ---")
            for movie in movies:
                print(f"    - {movie}")
            print("\n")
            
    else:
        print("No director has more than one movie in the scanned folders.")

    print("\n" + "="*50)
    print("RAW ANALYSIS DETAILS (first 100)")
    print("="*50)
    display(df_raw.head(100))

else:
    print(f"Error: The specified path does not exist: {MOVIES_FOLDER_PATH}")